# 🧙 Gandalf Chatbot (Fully Local)
This notebook runs a LangChain RAG chatbot locally using `transformers`, `sentence-transformers`, and `FAISS`. No cloud APIs or internet needed after model download.

In [ ]:
# 📦 Install Required Libraries
!pip install -q transformers sentence-transformers langchain faiss-cpu pypdf python-dotenv

In [ ]:
# 🔐 Load Environment Variables
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 📚 Load and Split PDF
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("lotr_book.pdf")  # Change to your PDF path
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(pages)
print(f"✅ Loaded and split {len(docs)} chunks.")

In [ ]:
# 🔎 Embed Chunks and Save Vectorstore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("gandalf_index")
print("✅ Vectorstore saved.")

In [ ]:
# 🤖 Load Local LLM with Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.1"  # Or other instruct-tuned LLM

print("⏬ Loading model (first time may take a few minutes)...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
llm = HuggingFacePipeline(pipeline=pipe)
print("✅ LLM loaded locally.")

In [ ]:
# 🧠 Load Vectorstore and Run RAG
from langchain.chains import RetrievalQA

retriever = FAISS.load_local("gandalf_index", embeddings).as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

question = "What happened in the mines of Moria?"
result = qa_chain.invoke({"query": question})

print("🧙 Gandalf says:\n", result['result'])